![Lille](img/lille.jpeg)

# Avant-propos
___

Ce notebook utilise un petit jeu de données fictif, inspiré du marché immobilier lillois. Il a été créé pour apprendre pas à pas comment fonctionne la régression linéaire.

L’idée est de prédire le prix d’un logement en fonction d'une information simple, sa surface. C’est un exemple facile à comprendre, parfait pour s’initier à la modélisation.

# Introduction
___

Nous allons découvrir comment le machine learning peut être utilisé pour **prédire une valeur numérique** à partir de données d’entrée. C’est ce qu’on appelle un problème de **régression**.

Nous allons nous intéresser à un cas simple mais très concret :  

`Prédire le prix d’un bien immobilier en fonction de sa surface.`

Ce type de question est au cœur de nombreux usages du ML :
- prédire un salaire selon l’expérience,
- estimer la consommation d’énergie selon la taille d’un logement,
- estimer la valeur d’un véhicule selon son kilométrage…

Notre objectif n’est pas seulement d’obtenir une prédiction correcte, mais surtout de comprendre et expérimenter les étapes fondamentales d’un projet de **machine learning supervisé**.

💡 Le machine learning (ML) n'est pas uniquement choisir un modèle, lui donner des données, et obtenir des prédictions.
En réalité, un projet ML suit un processus rigoureux, avec plusieurs étapes interdépendantes. Ce processus n’est pas figé, mais il suit une logique que l’on retrouve dans tous les projets, quel que soit le type de données ou de modèle.

Ce notebook présente un **pipeline** (démarche structurée), composée de plusieurs **étapes clés**, que l’on retrouve dans la majorité des projets de **machine learning supervisé**.

| Étapes                        | Description                                                                                               |
|-------------------------------| ----------------------------------------------------------------------------------------------------------|
| 1 - Data Loading              | Chargement des données (CSV, base de données, API, etc.)                                                  |
| 2 - Exploration               | Analyse des données : types, statistiques, corrélations, visualisations                                   |
| 3 - Data preparation          | Suppression des doublons, traitement des valeurs manquantes, création de features (sur tout le dataset)   |
| 4 - Train/Test Split          | Séparation des données pour évaluation (généralement 80/20 ou 70/30)                                      |
| 5 - Preprocessing             | Encodage, standardisation, etc. (fit sur le train, puis transform sur le test)                            |
| 6 - Model Training            | Entraînement du modèle sur les données d’entraînement                                                     |
| 7 - Evaluation                | Mesure des performances : MAE, MSE, R², accuracy, confusion matrix, etc..                                 |

# Import des bibliothèques
___

On commence par importer toutes les bibliothèques dont nous allons avoir besoin dans ce projet.

- **Pandas** : pour charger les données, les explorer et les modifier
- **NumPy** : pour effectuer des calculs numériques, notamment sur les tableaux
- **Matplotlib.pyplot** : pour créer des visualisations (nuages de points, courbes, histogrammes...)
- **Scikit-learn (sklearn)** :
  - `LinearRegression` : pour entraîner un modèle de régression linéaire
  - `train_test_split` : pour séparer les données en jeu d'entraînement et de test
  - `mean_squared_error`, `r2_score` : pour évaluer la performance du modèle
  - `StandardScaler` : pour standardiser les données si nécessaire
- **Joblib** : pour sauvegarder ou recharger un modèle déjà entraîné

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib

# 1. 📚 Data loading
___

La première étape de tout projet de machine learning consiste à charger les données dans l’environnement de travail. Dans ce notebook, nous utilisons un fichier au format CSV. Pour cela, nous utilisons la fonction read_csv() fournie par la bibliothèque Pandas, qui permet de lire le fichier et de le transformer en **DataFrame**.

💡 Les données peuvent provenir de nombreuses sources différentes : des fichiers **Excel** (.xlsx), des fichiers **JSON** (.json), des formats optimisés comme **Parquet** (.parquet), ou encore directement de **BDD** ou d’**API**.

In [27]:
# Code

# 2. 🔍 Exploration
___

### 2.1 Inspection rapide du dataset

Une fois les données chargées, il est important de commencer par en examiner un **aperçu rapide** pour s’assurer que le fichier a bien été lu, que les colonnes sont correctement nommées et que les premières lignes ont du sens.

In [28]:
# Code

Nous utilisons ensuite `df.info()` pour obtenir un résumé de la structure du DataFrame.  
Cette commande permet de vérifier :
- le nombre de lignes
- le type de chaque colonne (entier, flottant, objet, etc.)
- et de repérer la présence de valeurs manquantes.

In [29]:
# Code

Nous utilisons ensuite `df.describe()` pour obtenir un résumé statistique des variables numériques.  

Cette étape nous permet de mieux comprendre la structure des données : 
- on regarde les ordres de grandeur
- les valeurs typiques (comme la moyenne ou la médiane)
- on repère les éventuelles anomalies (comme des minimums ou maximums anormalement élevés).  

C’est une façon rapide de valider la cohérence globale du jeu de données avant d’aller plus loin.

In [30]:
# Code

### 2.2. Traitement des données manquantes

Pour traiter les valeurs manquantes nous devons choisir une stratégie adaptée : 
- soit les supprimer si elles sont peu nombreuses et non critiques (`.dropna()`)
- soit les remplacer par une valeur par défaut (comme la moyenne ou la médiane), selon le contexte (`.filla()`)

In [31]:
# Code

### 2.3. Anlayses univariées et bivariées

Maintenant que nous avons un aperçu général de notre dataset, nous allons passer à l’analyse exploratoire proprement dite  (EDA : **Exploratory Data Analysis**), en combinant des analyses **univariées** et **bivariées**.

L’**analyse univariée** consiste à étudier **une variable à la fois**. Elle permet de :
- **comprendre la distribution d’une variable**
- **repérer des tendances**
- **repérer des valeurs atypiques** (outliers)
- **détecter des incohérences** (par exemple : pas de surface négative, prix trop faibles, etc.).

L’**analyse bivariée**, quant à elle, vise à **explorer la relation entre deux variables**. Elle permet par exemple de :
- visualiser l’**effet d’une variable** (comme la surface) sur une autre (comme le prix)
- **identifier des corrélations ou des dépendances**

#### a. Surface des biens immobiliers

Nous pouvons visualiser la variable `surface_m2` sous forme d’histogramme, ce qui nous permet de voir comment les biens sont répartis en fonction de leur surface.

In [32]:
# Code

Après un histogramme qui montre la distribution globale, le boxplot nous donne une synthèse statistique visuelle : médiane, étendue, quartiles, et surtout les valeurs qui sortent du cadre.
Les deux graphiques sont complémentaires, surtout quand on veut repérer rapidement d’éventuelles anomalies.

In [33]:
# Code

#### b. Prix des biens immobiliers

Nous visualisons ici la variable `prix_euros` à l’aide d’un histogramme, ce qui permet de comprendre comment les biens sont répartis selon leur prix.

In [34]:
# Code

In [35]:
# Code

#### c. Relation entre la surface et le prix des biens

Un **nuage de points (scatter plot)** permet de visualiser chaque bien comme une paire (surface, prix). Ce type de graphique permet de :
- identifier des tendances générales
- identifier des des groupements de données
- identifier d’éventuelles relations linéaires

Dans notre cas, on observe visuellement qu’une augmentation de la surface semble généralement associée à une augmentation du prix. 

⚠️ Il ne s’agit ici que d'une analyse bivariée visuelle. Ceci constitue une première étape avant d’évaluer plus formellement cette relation à l’aide d’un modèle ou d’un indicateur statistique.

In [36]:
# Code

#### d. Correlations

Afin d’explorer les **relations linéaires** entre les variables de notre jeu de données, nous calculons la matrice de corrélation de Pearson. Cette méthode permet d’estimer dans quelle mesure deux variables numériques **varient ensemble**.

La matrice ci-dessous met en évidence une corrélation très forte entre la surface en mètres carrés (surface_m2) et le prix en euros (prix_euros).

In [37]:
# Code

# 3. Data preparation
___

### Détection des valeurs aberrantes (outliers)

Avant d’entraîner un modèle, il est important d’identifier les **valeurs aberrantes**, c’est-à-dire des données qui sont très éloignées des autres et qui pourraient **fausser les résultats**.

Ici, nous utilisons une méthode simple et classique basée sur l’**écart interquartile (IQR)**.

In [38]:
# Code

In [39]:
# Code

# 4. Train/Test split
___

Avant d’entraîner notre modèle de régression, il est essentiel de séparer nos données en deux ensembles distincts :
- un **ensemble d’entraînement** (**train set**), qui sera utilisé pour **ajuster le modèle**,
- un **ensemble de test** (**test set**), qui servira à **évaluer les performances du modèle sur des données jamais vues**.

Dans notre cas, nous avons choisi une répartition classique de 80 % pour l’entraînement et 20 % pour le test.
Cette séparation permet de vérifier si notre modèle est capable de généraliser à de nouvelles données, et non simplement de mémoriser les exemples qu’il a vus.

In [40]:
# Code

# 5. Preprocessing
___

Avant d’entraîner un modèle, il est important de **mettre toutes les variables à la même échelle**.  
La standardisation consiste à transformer les données pour qu’elles aient une moyenne de 0 et un écart-type de 1.

Cela permet au modèle d’apprendre plus efficacement, surtout quand les variables n’ont pas les mêmes unités (par exemple : surface en m², prix en euros).

On utilise ici le `StandardScaler` pour **apprendre la moyenne et l’écart-type** sur les données d’entraînement, puis on applique cette transformation aux données d’entraînement et de test.

In [41]:
# Code

# 6. Model training
___

### a. Apprentissage

Nous allons maintenant **entraîner** notre modèle de régression linéaire **à partir des données d’entraînement** préalablement standardisées.  
Le modèle va apprendre la **relation** entre la surface du bien (`X_train_scaled`) et son prix (`y_train_scaled`) en ajustant une droite qui minimise l’erreur quadratique (**MSE**).

Cette étape est essentielle : c’est ici que le modèle **apprend** à prédire en se basant sur les exemples fournis.

In [42]:
# Code

### b. Visualisation du modèle entraîné

In [43]:
# Code

# 7. Evaluation
___

Après l’entraînement il est crucial d’évaluer le modèle sur un jeu de données qu’il n’a jamais vu : c’est ce qu’on appelle le **jeu de test**.  
L’objectif est de mesurer sa performance c'est à dire sa **capacité du modèle à généraliser** ses prédictions à de nouvelles données réelles.
On affiche ici :
- la **MSE** (Mean Squared Error) : l’erreur quadratique moyenne entre les prédictions et les vraies valeurs
- la **RMSE** (Root Mean Squared Error) : la racine carrée de la MSE, plus interprétable car dans la même unité que la variable cible
- le **R² score** : appelé aussi **coefficient de détermination**, est un indicateur de performance compris entre 0 et 1, qui mesure la proportion de la variance expliquée par le modèle

### a. Scores

In [44]:
# Code

### b. Visualisation des prédictions

In [45]:
# Code

# 8. Sauvegarde du modèle et des standardiseurs (scalers)
___

Une fois notre modèle entraîné, il est important de le sauvegarder pour pouvoir le réutiliser plus tard, **sans devoir le réentraîner**.  
Nous sauvegardons également les objets `StandardScaler` utilisés pour la standardisation, ⚠️ **car toute nouvelle donnée devra être transformée avec les mêmes paramètres (moyenne et écart-type) que ceux appris sur les données d’entraînement**.

La sauvegarde se fait ici avec la bibliothèque `joblib`, qui permet de **sérialiser des objets Python** dans des fichiers .pkl.

In [46]:
# Code

# Comment utiliser le modèle ?
___

In [47]:
# Code